In [1]:
# 2) Open closed principle. Classes should be open for extension/inheritance and closed for modifications

from enum import Enum


class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3


class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3


class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size

## old way /Antipattern. Modify SAME class by adding additional methods
## Each time you want a new filter for products. making the productFilter
## A God class of sorts 

class ProductFilter:
    ## given list of products and color yields those that have the same color
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p
## given list of products and size yields those that have the same size
    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p
## given list of products and (color,size) yields those that have the same color and size
    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.color == color and p.size == size:
                yield p

    # state space explosion..list of new fitlers can go on and on and the class explodes!
    # 3 criteria
    # c s w cs sw cw csw = 7 methods

    # OCP = open for extension, closed for modification

############################################################
# Enterprise Design Pattern 
####################################

## Build Abstract class that provide the interface for defining is_satified spec
class Specification:
    def is_satisfied(self, item):
        pass

    # and operator makes life easier (to define a combinator spec to combine two different specs)
    ##  a spec is just implementation for is_satisfied based on some criteria! 
    def __and__(self, other):
        return AndSpecification(self, other)

## Abstract class for the filter method that apply a spec class on a list of items
class Filter:
    def filter(self, items, spec):
        pass


class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color

    def is_satisfied(self, item):
        return item.color == self.color


class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size

    def is_satisfied(self, item):
        return item.size == self.size


class AndSpecification(Specification):
    def __init__(self, spec1, spec2):
        self.spec2 = spec2
        self.spec1 = spec1

    def is_satisfied(self, item):
        return self.spec1.is_satisfied(item) and \
               self.spec2.is_satisfied(item)


class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item


apple = Product('Apple', Color.GREEN, Size.SMALL)
tree = Product('Tree', Color.GREEN, Size.LARGE)
house = Product('House', Color.BLUE, Size.LARGE)

products = [apple, tree, house]

pf = ProductFilter()
print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
    print(f' - {p.name} is green')

# ^ BEFORE

# v AFTER
bf = BetterFilter()

print('Green products (new):')
green = ColorSpecification(Color.GREEN)
for p in bf.filter(products, green):
    print(f' - {p.name} is green')

print('Large products:')
large = SizeSpecification(Size.LARGE)
for p in bf.filter(products, large):
    print(f' - {p.name} is large')

print('Large blue items:')
# large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))
large_blue = large and ColorSpecification(Color.BLUE)
for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue')

Green products (old):
 - Apple is green
 - Tree is green
Green products (new):
 - Apple is green
 - Tree is green
Large products:
 - Tree is large
 - House is large
Large blue items:
 - House is large and blue


In [2]:
large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))


In [5]:
large_blue.spec1.size

<Size.LARGE: 3>

In [6]:
large_blue.spec2.color

<Color.BLUE: 3>